# Procesamiento de datos

In [1]:
import os

import librosa
from librosa import display

import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import IPython
import IPython.display as ipd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

### variables de entorno

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
DATASET = '/content/drive/MyDrive/ML Proyecto/DataSets/Emotions'
LABELS = ['Angry', 'Happy', 'Sad', 'Fearful', 'Disgusted', 'Neutral', 'Suprised']
SAMPLE_RATE = 22050

### funciones de procesamiento

In [10]:
def remove_silence(signal, threshold=0.005):
    """
    Remove silence at the beginning and at the end of the signal
    """
    for i in range(len(signal)):
        if np.abs(signal[i]) > threshold:
            break
    for j in range(len(signal)-1, 0, -1):
        if np.abs(signal[j]) > threshold:
            break
    return signal[i:j]

In [11]:
resize = int(SAMPLE_RATE*2)
print(resize)
def signal_resize(signal, length=resize):
    """
    Cut the signal to the given length, or pad it with zeros if it is shorter
    """
    if len(signal) > length:
        return signal[:length]
    else:
        return np.pad(signal, (0, max(0, length - len(signal))), "constant")

44100


### Read the dataset

In [35]:
emotions = []

for dirname, dirs, filenames in os.walk(DATASET):
    print(dirname)
    for filename in filenames[:10]:
        path = os.path.join(dirname, filename)
        title = dirname.split('/')[-1]
        category = LABELS.index(title)

        signal, sample_rate = librosa.load(path, sr=SAMPLE_RATE)
        signal = remove_silence(signal)
        mean = np.mean(signal)
        std = np.std(signal) # Standard deviation

        signal = signal_resize(signal)
        time = len(signal) / sample_rate
        zero_crossings = sum(librosa.zero_crossings(signal, pad=False))
        zero_crossing_rate = librosa.feature.zero_crossing_rate(signal)[0]
        stft_mag, stft_phase = librosa.magphase(librosa.stft(signal, n_fft=512)) # Short-time Fourier transform, n_fft = 512 for 22050 Hz
        #print(stft_mag.shape)
        freqs = librosa.fft_frequencies(sr=sample_rate)
        max_stft_freq = freqs[int(np.argmax(stft_mag)/stft_mag.shape[1])]
        min_stft_freq = freqs[int(np.argmin(stft_mag)/stft_mag.shape[1])]
        stft_phase = np.angle(stft_phase)
        rms = librosa.feature.rms(S=stft_mag, frame_length=512)[0] # Root mean square - Energy # FRAME LENG = 512 
        mfcc = librosa.feature.mfcc(S=stft_mag, sr=sample_rate) # Mel-frequency cepstral coefficients

        emotions.append((
            path,
            title,
            category, 
            signal, 
            mean, 
            std, 
            sample_rate, 
            time, 
            zero_crossings, 
            zero_crossing_rate, 
            stft_mag, 
            stft_phase, 
            max_stft_freq,
            min_stft_freq,
            rms,
            mfcc,
        ))

/content/drive/MyDrive/ML Proyecto/DataSets/Emotions
/content/drive/MyDrive/ML Proyecto/DataSets/Emotions/Fearful
/content/drive/MyDrive/ML Proyecto/DataSets/Emotions/Angry
/content/drive/MyDrive/ML Proyecto/DataSets/Emotions/Sad
/content/drive/MyDrive/ML Proyecto/DataSets/Emotions/Disgusted
/content/drive/MyDrive/ML Proyecto/DataSets/Emotions/Happy
/content/drive/MyDrive/ML Proyecto/DataSets/Emotions/Neutral
/content/drive/MyDrive/ML Proyecto/DataSets/Emotions/Suprised


In [36]:
columns = [
    'path',
    'title',
    'category', 
    'signal', 
    'mean',
    'std',
    'sample_rate', 
    'time', 
    'zero_crossings', 
    'zero_crossing_rate',
    'stft_mag',
    'stft_phase',
    'max_stft_freq',
    'min_stft_freq',
    'rms',
    'mfcc',
]

In [37]:
emotions_frame = pd.DataFrame(emotions, columns=columns)
emotions_frame.head()

,path,title,category,signal,mean,std,sample_rate,time,zero_crossings,zero_crossing_rate,stft_mag,stft_phase,max_stft_freq,min_stft_freq,rms,mfcc
0,/content/drive/MyDrive/ML Proyecto/DataSets/Em...,Fearful,3,"[0.005596205, 0.0069596493, 0.007269218, 0.007...",0.000007,0.079327,22050,2.0,2828,"[0.01904296875, 0.02783203125, 0.03125, 0.0317...","[[0.26757145, 0.312826, 0.22310682, 0.32838503...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.1415925, -3...",204.565430,2756.250000,"[0.0030986406318568595, 0.002580976869864057, ...","[[0.30927026, 0.30343258, 0.27390563, 0.336039..."
1,/content/drive/MyDrive/ML Proyecto/DataSets/Em...,Fearful,3,"[0.005551721, 0.005384678, 0.0054556574, 0.006...",-0.000017,0.025047,22050,2.0,2129,"[0.015625, 0.02197265625, 0.029296875, 0.04052...","[[1.2640425, 1.0543555, 0.48817363, 0.08353723...","[[0.0, 0.0, 0.0, 0.0, -3.1415925, -3.1415925, ...",118.432617,2756.250000,"[0.0047171375140426484, 0.003851546763079716, ...","[[0.40311715, 0.34826288, 0.318527, 0.27663544..."
2,/content/drive/MyDrive/ML Proyecto/DataSets/Em...,Fearful,3,"[-0.005505172, -0.005133936, -0.005757561, -0....",-0.000037,0.050874,22050,2.0,2611,"[0.0107421875, 0.02197265625, 0.03076171875, 0...","[[0.098904565, 0.3204608, 0.650316, 0.6148809,...","[[0.0, 0.0, 0.0, 0.0, 0.0, -3.1415925, -3.1415...",150.732422,2756.250000,"[0.00359094080980228, 0.003680727614284965, 0....","[[0.26380718, 0.30610663, 0.31794986, 0.353068..."
3,/content/drive/MyDrive/ML Proyecto/DataSets/Em...,Fearful,3,"[0.0051507554, 0.005338836, 0.005147377, 0.005...",-0.000048,0.015225,22050,2.0,1935,"[0.01123046875, 0.0234375, 0.0283203125, 0.030...","[[0.3696451, 0.22000863, 0.017495703, 0.125324...","[[-3.1415925, -3.1415925, -3.1415925, -3.14159...",43.066406,2756.250000,"[0.0038691303950421957, 0.003722841487925813, ...","[[0.31272984, 0.32106757, 0.28703526, 0.282338..."
4,/content/drive/MyDrive/ML Proyecto/DataSets/Em...,Fearful,3,"[-0.00566154, -0.0056041353, -0.005215304, -0....",-0.000061,0.061859,22050,2.0,4324,"[0.02001953125, 0.03369140625, 0.04443359375, ...","[[1.2423474, 0.7442686, 0.0713801, 0.19598104,...","[[-3.1415925, -3.1415925, 0.0, 0.0, 0.0, 0.0, ...",96.899414,2164.086914,"[0.003407469657889043, 0.0033695092412063636, ...","[[0.29737628, 0.3025842, 0.26858005, 0.2346263..."


In [38]:
emotions_frame['mfcc'].apply(lambda x : x.flatten())

0     [0.30927026, 0.30343258, 0.27390563, 0.3360392...
1     [0.40311715, 0.34826288, 0.318527, 0.27663544,...
2     [0.26380718, 0.30610663, 0.31794986, 0.3530688...
3     [0.31272984, 0.32106757, 0.28703526, 0.2823386...
4     [0.29737628, 0.3025842, 0.26858005, 0.23462635...
                            ...                        
65    [0.36032087, 0.37025434, 0.3599007, 0.31283215...
66    [0.5168242, 0.6716985, 0.7243828, 0.85607237, ...
67    [0.5557998, 0.49031964, 0.34778598, 0.41059193...
68    [0.32468295, 0.38264218, 0.39511186, 0.3871868...
69    [0.4064712, 0.42737195, 0.4663248, 0.46622393,...
Name: mfcc, Length: 70, dtype: object

In [39]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

In [79]:
lst = []
for _, row in emotions_frame.iterrows():
  lst.append((row['mfcc'].flatten(), row['category']))

In [74]:
# Creating X and y: zip makes a list of all the first elements, and a list of all the second elements.
X, y = zip(*lst)
import numpy as np
X = np.asarray(X)
y = np.asarray(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

x_trainn = np.expand_dims(X_train, axis=2) 
x_testnn = np.expand_dims(X_test, axis=2)

In [75]:
x_trainn.shape

(56, 6900, 1)

In [82]:
model = keras.Sequential([

        # input layer
        keras.layers.Flatten(input_shape=(x_trainn.shape[1], 1)),

        # 1st dense layer
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.1)),
        keras.layers.Dropout(0.5),

        # 2nd dense layer
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.5),

        # 3rd dense layer
        keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

        # output layer
        keras.layers.Dense(10, activation='softmax')
    ])

    # compile model
optimiser = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [83]:
history = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
1/1 [==============================] - 1s 943ms/step - loss: 56.3704 - accuracy: 0.1071 - val_loss: 55.1100 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 54ms/step - loss: 56.6979 - accuracy: 0.0357 - val_loss: 55.0668 - val_accuracy: 0.0714
Epoch 3/100
1/1 [==============================] - 0s 57ms/step - loss: 56.2240 - accuracy: 0.0714 - val_loss: 55.0251 - val_accuracy: 0.0714
Epoch 4/100
1/1 [==============================] - 0s 61ms/step - loss: 56.8207 - accuracy: 0.0714 - val_loss: 54.9830 - val_accuracy: 0.0714
Epoch 5/100
1/1 [==============================] - 0s 62ms/step - loss: 57.2962 - accuracy: 0.0714 - val_loss: 54.9393 - val_accuracy: 0.0714
Epoch 6/100
1/1 [==============================] - 0s 71ms/step - loss: 56.3228 - accuracy: 0.1429 - val_loss: 54.8940 - val_accuracy: 0.0714
Epoch 7/100
1/1 [==============================] - 0s 71ms/step - loss: 57.0129 - accuracy: 0.1071 - val_loss: 54.8545 - val_accuracy: 0.0714
E